In [1]:
%pip install -q datasets

Note: you may need to restart the kernel to use updated packages.


In [2]:
import math

In [3]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"

# [1] Causal language modeling
- [источник](https://huggingface.co/docs/transformers/en/tasks/language_modeling)

Существует два типа языкового моделирования - каузальное и маскированное. В данном руководстве рассматривается каузальное языковое моделирование.

In [4]:
#@title
from IPython.display import HTML

HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/Vpjb1lu0MDk?rel=0&amp;controls=0&amp;showinfo=0" frameborder="0" allowfullscreen></iframe>')

/home/dasha/work/testing/eval-env/lib/python3.9/site-packages/IPython/core/display.py:431: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


Казуальное языковое моделирование (causal language modeling) — это предсказание следующего токена в последовательности, при котором модель может «видеть» только токены, находящиеся слева. Иными словами, модель не имеет доступа к будущим токенам. Примером такой модели является GPT-2.

В этом руководстве мы узнаем, как:

1. Дообучить модель [DistilGPT2](https://huggingface.co/distilgpt2) на датасете [ELI5](https://huggingface.co/datasets/eli5).  
2. Использовать дообученную модель для генерации текста.

<Tip>  
Мы можем дообучать и другие архитектуры для задач казуального языкового моделирования, следуя тем же шагам, что описаны в этом руководстве.  
Можно выбрать одну из следующих архитектур:

<!--This tip is automatically generated by `make fix-copies`, do not fill manually!-->
[BART](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/bart), [BERT](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/bert), [Bert Generation](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/bert-generation), [BigBird](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/big_bird), [BigBird-Pegasus](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/bigbird_pegasus), [BioGpt](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/biogpt), [Blenderbot](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/blenderbot), [BlenderbotSmall](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/blenderbot-small), [BLOOM](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/bloom), [CamemBERT](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/camembert), [CodeGen](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/codegen), [CPM-Ant](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/cpmant), [CTRL](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/ctrl), [Data2VecText](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/data2vec-text), [ELECTRA](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/electra), [ERNIE](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/ernie), [GIT](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/git), [GPT-Sw3](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/gpt-sw3), [OpenAI GPT-2](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/gpt2), [GPTBigCode](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/gpt_bigcode), [GPT Neo](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/gpt_neo), [GPT NeoX](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/gpt_neox), [GPT NeoX Japanese](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/gpt_neox_japanese), [GPT-J](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/gptj), [LLaMA](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/llama), [Marian](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/marian), [mBART](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/mbart), [MEGA](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/mega), [Megatron-BERT](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/megatron-bert), [MVP](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/mvp), [OpenLlama](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/open-llama), [OpenAI GPT](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/openai-gpt), [OPT](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/opt), [Pegasus](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/pegasus), [PLBart](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/plbart), [ProphetNet](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/prophetnet), [QDQBert](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/qdqbert), [Reformer](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/reformer), [RemBERT](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/rembert), [RoBERTa](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/roberta), [RoBERTa-PreLayerNorm](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/roberta-prelayernorm), [RoCBert](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/roc_bert), [RoFormer](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/roformer), [RWKV](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/rwkv), [Speech2Text2](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/speech_to_text_2), [Transformer-XL](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/transfo-xl), [TrOCR](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/trocr), [XGLM](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/xglm), [XLM](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/xlm), [XLM-ProphetNet](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/xlm-prophetnet), [XLM-RoBERTa](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/xlm-roberta), [XLM-RoBERTa-XL](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/xlm-roberta-xl), [XLNet](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/xlnet), [X-MOD](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/xmod)


<!--End of the generated tip-->

</Tip>

Перед началом убедитесь, что у вас установлены все необходимые библиотеки:

```bash
pip install transformers datasets evaluate
```

## Load ELI5 dataset

Начнем с загрузки небольшого поднабора раздела r/askscience из датасета ELI5 с помощью библиотеки 🤗 Datasets.  
Это позволит поэкспериментировать и убедиться, что всё работает корректно, прежде чем тратить время на обучение на полном датасете.

In [5]:
from datasets import load_dataset

eli5 = load_dataset("sentence-transformers/eli5", split="train[:5000]")

/home/dasha/work/testing/eval-env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Разделим подвыборку train_asks датасета на обучающую и тестовую части с помощью метода train_test_split:

In [6]:
eli5 = eli5.train_test_split(test_size=0.2)

Посмотрим на отдельный элемент датасета:

In [7]:
eli5["train"][0]

{'question': 'When does a date become a girl/boyfriend?',
 'answer': "Probably when the individuals involved start calling each other boyfriend/girlfriend. I don't think there's a solid definition beyond that."}

В данном случае нам интересно поле `answer`. В задачах языкового моделирвания нам не нужны метки (фактически мы обучаем модель в unsupervised режиме), потому что следующее слово _является_ меткой.

## Предобработка

In [8]:
#@title
from IPython.display import HTML

HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/ma1TrR7gE7I?rel=0&amp;controls=0&amp;showinfo=0" frameborder="0" allowfullscreen></iframe>')

/home/dasha/work/testing/eval-env/lib/python3.9/site-packages/IPython/core/display.py:431: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


Следующим шагом будет загрузка DistilGPT2 для обработки поля `answer`:

In [9]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilgpt2")

Напишем функцию токенизации ответа

In [10]:
def preprocess_function(examples):
    return tokenizer(examples["answer"])

Чтобы применить эту функцию предобработки ко всему датасету, используем метод [`map`](https://huggingface.co/docs/datasets/main/en/package_reference/main_classes#datasets.Dataset.map) из библиотеки 🤗 Datasets. Мы можем ускорить выполнение `map`, установив `batched=True` для пакетной обработки нескольких элементов одновременно и увеличив число процессов с помощью параметра `num_proc`. Удалим все ненужные столбцы.

In [11]:
tokenized_eli5 = eli5.map(
    preprocess_function,
    batched=True,
    num_proc=4,
    remove_columns=eli5["train"].column_names,
)

Map (num_proc=4): 100%|██████████| 1000/1000 [00:00<00:00, 4942.23 examples/s]


In [12]:
tokenized_eli5

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 4000
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 1000
    })
})

Этот датасет содержит последовательности токенов, но некоторые из них длиннее максимальной допустимой длины входа для модели.

Теперь мы можем использовать вторую функцию предобработки, чтобы:

- объединить все последовательности в одну,
- разбить полученную объединённую последовательность на более короткие фрагменты, определяемые параметром `block_size`, который должен быть короче максимальной длины входа и достаточно мал, чтобы поместиться в память графического процессора.

In [13]:
block_size = 128


def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
    # customize this part to your needs.
    if total_length >= block_size:
        total_length = (total_length // block_size) * block_size
    # Split by chunks of block_size.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

Применим функцию `group_texts` ко всему набору данных:

In [14]:
lm_dataset = tokenized_eli5.map(group_texts, batched=True, num_proc=4)

Map (num_proc=4): 100%|██████████| 1000/1000 [00:00<00:00, 5988.47 examples/s]


Теперь создадим батч (пакет) примеров с помощью класса [`DataCollatorForLanguageModeling`](https://huggingface.co/docs/transformers/main/en/main_classes/data_collator#transformers.DataCollatorForLanguageModeling).  

Используем токен конца последовательности в качестве токена для паддинга и установим `mlm=False`. Это означает, что входы будут использоваться как метки, сдвинутые вправо на один элемент.

In [15]:
from transformers import DataCollatorForLanguageModeling

tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

## Train

<Tip>

Если вы ещё не знакомы с дообучением модели с помощью класса [`Trainer`](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer), загляните в [базовый туториал](https://huggingface.co/docs/transformers/main/en/training#train-with-pytorch-trainer)!

</Tip>

Теперь мы готовы начать обучение модели! Загрузим DistilGPT2 с помощью [`AutoModelForCausalLM`](https://huggingface.co/docs/transformers/main/en/model_doc/auto#transformers.AutoModelForCausalLM):

In [16]:
from transformers import AutoModelForCausalLM, TrainingArguments, Trainer

model = AutoModelForCausalLM.from_pretrained("distilgpt2")
model = model.to(device)

In [17]:
import wandb
wandb.init(mode="disabled")

In [21]:
training_args = TrainingArguments(
    output_dir="my_awesome_eli5_clm-model",
    eval_strategy="epoch",
    num_train_epochs=5,
    learning_rate=2e-5,
    weight_decay=0.01,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_dataset["train"],
    eval_dataset=lm_dataset["test"],
    data_collator=data_collator,
)

eval_results = trainer.evaluate()
print(f'Perplexity: {math.exp(eval_results["eval_loss"]):.2f}')

trainer.train()

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Perplexity: 61.16


Epoch,Training Loss,Validation Loss,Model Preparation Time
1,No log,3.889793,0.001600
2,3.982000,3.883291,0.001600
3,3.844100,3.885188,0.001600
4,3.844100,3.884944,0.001600
5,3.784700,3.888149,0.001600


TrainOutput(global_step=1755, training_loss=3.854017921730324, metrics={'train_runtime': 305.5124, 'train_samples_per_second': 45.956, 'train_steps_per_second': 5.744, 'total_flos': 458575797288960.0, 'train_loss': 3.854017921730324, 'epoch': 5.0})

In [22]:
trainer.save_model("my_awesome_eli5_clm-model")
tokenizer.save_pretrained("my_awesome_eli5_clm-model")

('my_awesome_eli5_clm-model/tokenizer_config.json',
 'my_awesome_eli5_clm-model/special_tokens_map.json',
 'my_awesome_eli5_clm-model/vocab.json',
 'my_awesome_eli5_clm-model/merges.txt',
 'my_awesome_eli5_clm-model/added_tokens.json',
 'my_awesome_eli5_clm-model/tokenizer.json')

После завершения обучения используем метод [evaluate()](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer.evaluate), чтобы оценить нашу модель и получить ее перплексию:

In [23]:


eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

Perplexity: 48.82


## Инференс

In [24]:
prompt = "Tokenization in NLP is a crucial preprocessing step that involves"

In [25]:
from transformers import pipeline

generator = pipeline("text-generation", model="my_awesome_eli5_clm-model", device=device)

Device set to use cuda


In [26]:
result = generator(prompt)
result

[{'generated_text': 'Tokenization in NLP is a crucial preprocessing step that involves the processing of a number of bits of data and making it transparent. This is a big deal to me because, like how the world works for a computer, that number of binary n'}]

In [27]:
result[0]['generated_text']

'Tokenization in NLP is a crucial preprocessing step that involves the processing of a number of bits of data and making it transparent. This is a big deal to me because, like how the world works for a computer, that number of binary n'

Токенизируем текст и получим `input_ids` - тензоры PyTorch:

In [28]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("my_awesome_eli5_clm-model")
inputs = tokenizer(prompt, return_tensors="pt").input_ids

Используем метод [`generate()`](https://huggingface.co/docs/transformers/main/en/main_classes/text_generation#transformers.GenerationMixin.generate) для генерации текста.  
Чтобы узнать больше о различных стратегиях генерации текста и параметрах, управляющих процессом генерации, см. страницу [Стратегии генерации текста](https://huggingface.co/docs/transformers/main/en/generation_strategies).

In [29]:
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained("my_awesome_eli5_clm-model")
outputs = model.generate(inputs, max_new_tokens=100, do_sample=True, top_k=50, top_p=0.95)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Перекодируем сгенерированные идентификаторы токенов обратно в текст с помощью метода decode:

In [30]:
tokenizer.batch_decode(outputs, skip_special_tokens=True)

['Tokenization in NLP is a crucial preprocessing step that involves some of the highest levels of compression and compression in the media. This process also involves an unencumbered process called the processing of audio files that are either converted in HD, or converted into full HD. This process allows each file to be compressed by a process called transcribe. Each of these processes can be converted to digital and audio (decoders and transcrites) and then transferred to another process called transcribe to have the same amount of time, but still be processed in the same format']